# Onebox helper

**Usage profile:**
- start search center
- preferred user language

**Results limits:**
```
OneBoxMap.default_terms_limit = 3
OneBoxMap.default_results_limit = 20
OneBoxMap.default_suggestions_limit = 5
```

In [ ]:
from search.widgets import OneBoxMap, Design
OneBoxMap(language='en', latitude=40.71455, longitude=-74.00714, design=Design.three, output_format='text').run()

# Json with Map

`Design.two`

In [ ]:
OneBoxMap(language='en', latitude=40.71455, longitude=-74.00714, design=Design.two, output_format='json').run()